## Complete Step-by-Step Solution: At-Risk Member Prediction & Causal Analytics
-Problem Statement
-Predict members who will make high claims OR develop high-risk conditions in the future (3-12 months   ahead), and identify root causes for actionable interventions.

📋 STEP-BY-STEP SOLUTION ROADMAP
-- PHASE 1: DATA PREPARATION & UNDERSTANDING

STEP 1: Define "At-Risk" Members Clearly
1.1 Business Definition Workshop

In [1]:
# Define what constitutes "at-risk" - get stakeholder agreement

AT_RISK_CRITERIA = {
    'high_claim': {
        'threshold_type': 'percentile',  # or absolute amount
        'threshold_value': 90,  # 90th percentile
        'time_window': '6_months',  # future prediction window
        'min_amount': 1000  # £5,000 minimum
    },
    'high_risk_condition': {
        'conditions': [
            'Cancer',
            'Cardiovascular Disease', 
            'Chronic Kidney Disease',
            'Diabetes Complications',
            'Mental Health Crisis',
            'Musculoskeletal Chronic'
        ],
        'severity_threshold': 'requires_ongoing_treatment'
    }
}

## 1.2 Create Ground Truth Labels

In [3]:
import pandas as pd

In [4]:
claims_df = pd.read_csv("uk_pmi_claims_200k.csv")

In [5]:
membership_df = pd.read_csv("uk_pmi_membership_120k.csv")

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def create_target_labels(claims_df, membership_df, observation_date, prediction_window_months=6):
    """
    Create target variables for at-risk prediction
    
    Parameters:
    - observation_date: Date at which we make prediction (e.g., '2025-01-01')
    - prediction_window_months: How far ahead to predict (default 6 months)
    """
    
    observation_date = pd.to_datetime(observation_date)
    future_end_date = observation_date + timedelta(days=30*prediction_window_months)
    
    # Target 1: High Claim in Future Window
    future_claims = claims_df[
        (pd.to_datetime(claims_df['Paid Date']) > observation_date) &
        (pd.to_datetime(claims_df['Paid Date']) <= future_end_date)
    ]
    
    future_claim_amounts = future_claims.groupby('Claimant Unique ID')['Claim Amount'].sum()
    high_claim_threshold = future_claim_amounts.quantile(0.90)
    
    target_df = membership_df[['Unique ID']].copy()
    target_df['future_total_claims'] = target_df['Unique ID'].map(future_claim_amounts).fillna(0)
    target_df['will_make_high_claim'] = (target_df['future_total_claims'] > high_claim_threshold).astype(int)
    
    # Target 2: High-Risk Condition Development
    high_risk_conditions = [
        'Cancer', 'Cardiovascular', 'Chronic Kidney', 
        'Diabetes', 'Mental Health', 'Chronic Pain'
    ]
    
    future_conditions = future_claims.groupby('Claimant Unique ID')['Condition Category'].apply(
        lambda x: any(condition in str(x).upper() for condition in [c.upper() for c in high_risk_conditions])
    )
    
    target_df['will_develop_high_risk_condition'] = target_df['Unique ID'].map(future_conditions).fillna(0).astype(int)
    
    # Combined Target: At-Risk Member (either high claim OR high-risk condition)
    target_df['is_at_risk'] = (
        (target_df['will_make_high_claim'] == 1) | 
        (target_df['will_develop_high_risk_condition'] == 1)
    ).astype(int)
    
    # Additional: Risk severity score (0-100)
    target_df['risk_severity_score'] = (
        0.6 * target_df['will_make_high_claim'] + 
        0.4 * target_df['will_develop_high_risk_condition']
    ) * 100
    
    return target_df

# Execute
observation_date = '2024-06-01'  # Use as your "present" for prediction
targets = create_target_labels(claims_df, membership_df, observation_date)

print("Target Distribution:")
print(targets['is_at_risk'].value_counts(normalize=True))
print(f"\nAt-risk rate: {targets['is_at_risk'].mean()*100:.2f}%")


Target Distribution:
is_at_risk
0    0.980267
1    0.019733
Name: proportion, dtype: float64

At-risk rate: 1.97%


## STEP 2: Data Integration & Temporal Alignment
#### 2.1 Create Point-in-Time Features

In [13]:
def create_point_in_time_dataset(claims_df, membership_df, observation_date, lookback_months=24):
    """
    Create features using ONLY data available before observation_date
    This prevents data leakage
    """
    
    observation_date = pd.to_datetime(observation_date)
    lookback_start = observation_date - timedelta(days=30*lookback_months)
    
    # Filter historical claims (before observation date)
    historical_claims = claims_df[
        pd.to_datetime(claims_df['Paid Date']) < observation_date
    ].copy()
    
    # Filter to lookback window
    historical_claims = historical_claims[
        pd.to_datetime(historical_claims['Paid Date']) >= lookback_start
    ]
    
    print(f"Historical claims: {len(historical_claims)} records")
    print(f"Date range: {historical_claims['Paid Date'].min()} to {historical_claims['Paid Date'].max()}")
    
    return historical_claims, membership_df

# Execute
observation_date = '2024-06-01'
historical_claims, membership = create_point_in_time_dataset(
    claims_df, membership_df, observation_date, lookback_months=60
)

Historical claims: 148054 records
Date range: 2020-01-23 to 2024-05-31


#### 2.2 Join Tables with Validation

In [ ]:
def join_claims_membership(claims_df, membership_df):
    """
    Join claims and membership with data quality checks
    """
    
    # Check for duplicates
    print("=== Data Quality Checks ===")
    print(f"Duplicate Claim IDs: {claims_df['Claim ID'].duplicated().sum()}")
    print(f"Duplicate Member IDs: {membership_df['Unique ID'].duplicated().sum()}")
    
    # Join
    df = membership_df.merge(
        claims_df,
        left_on='Unique ID',
        right_on='Claimant Unique ID',
        how='left',
        indicator=True
    )
    
    print(f"\nJoin Statistics:")
    print(df['_merge'].value_counts())
    
    # Identify members with no claims
    members_no_claims = df[df['_merge'] == 'left_only']['Unique ID'].nunique()
    print(f"\nMembers with no claims in period: {members_no_claims}")
    
    return df

df = join_claims_membership(historical_claims, membership)

=== Data Quality Checks ===
Duplicate Claim IDs: 0
Duplicate Member IDs: 0

Join Statistics:
_merge
both          148054
left_only      87143
right_only         0
Name: count, dtype: int64

Members with no claims in period: 87143


### STEP 3: Comprehensive Feature Engineering
#### 3.1 Demographic & Membership Features

In [22]:
df.drop(['Client Name_y', 'Client Identifier_y',
       'Scheme Category/ Section Name_y',
       'Scheme Category/ Section Name Identifier_y', 'Status of Member_y','Unique Member Reference_y', 'Contract Start Date_y',
       'Contract End Date_y'],axis=1,inplace=True)

In [23]:
df.columns = [col[:-2] if col.endswith('_x') else col for col in df.columns]


In [24]:
df.columns

Index(['Client Name', 'Client Identifier', 'Scheme Category/ Section Name',
       'Scheme Category/ Section Name Identifier', 'Unique Member Reference',
       'Unique ID', 'Status of Member', 'Status of Registration',
       'Year of Birth', 'Gender', 'Short Post Code of Member',
       'Contract Start Date', 'Contract End Date', 'Original Date of Joining',
       'Scheme Policy Joining Date', 'Registration Status', 'Lapse Date',
       'Claimant Unique ID', 'Claimant Year of Birth', 'Claimant Gender',
       'Short Post Code', 'Claim ID', 'Incurred Date', 'Paid Date',
       'Condition Code', 'Impairment Code', 'Condition Category',
       'Treatment Type', 'Claim Type', 'Ancillary Service Type',
       'Treatment Location', 'Provider Type', 'Admission Date',
       'Discharge Date', 'Calculate Length of Service', 'Claim Amount',
       'Amount Paid', '_merge'],
      dtype='object')

In [28]:
def create_demographic_features(df, observation_date):
    """
    Create demographic and membership-based features
    """
    
    observation_date = pd.to_datetime(observation_date)
    features = df[['Unique ID']].drop_duplicates().copy()
    
    # Age calculation
    features = features.merge(
        df.groupby('Unique ID')['Year of Birth'].first(),
        on='Unique ID'
    )
    features['age'] = observation_date.year - features['Year of Birth']
    
    # Age groups
    features['age_group'] = pd.cut(
        features['age'], 
        bins=[0, 25, 35, 45, 55, 65, 100],
        labels=['<25', '25-34', '35-44', '45-54', '55-64', '65+']
    )
    
   # Merge gender
    features = features.merge(
        df.groupby('Unique ID')['Gender'].first(),
        on='Unique ID'
    )

    # One-hot encode all gender categories
    gender_dummies = pd.get_dummies(features['Gender'], prefix='gender')

    # Combine with main dataset
    features = pd.concat([features, gender_dummies], axis=1)

    # Optionally drop original gender column
    features.drop(columns=['Gender'], inplace=True)
    
    # Membership tenure
    features = features.merge(
        df.groupby('Unique ID')['Original Date of Joining'].first(),
        on='Unique ID'
    )
    features['Original Date of Joining'] = pd.to_datetime(features['Original Date of Joining'])
    features['membership_tenure_days'] = (observation_date - features['Original Date of Joining']).dt.days
    features['membership_tenure_years'] = features['membership_tenure_days'] / 365.25
    
    # Membership status
    features = features.merge(
        df.groupby('Unique ID')['Registration Status'].first(),
        on='Unique ID'
    )
    features['is_active_member'] = (features['Registration Status'] == 'Active').astype(int)
    
    # Scheme information
    features = features.merge(
        df.groupby('Unique ID')['Scheme Category/ Section Name'].first(),
        on='Unique ID'
    )
    
    # Contract duration
    df['Contract Start Date'] = pd.to_datetime(df['Contract Start Date'], errors='coerce')
    df['Contract End Date'] = pd.to_datetime(df['Contract End Date'], errors='coerce')
    
    contract_info = df.groupby('Unique ID').agg({
        'Contract Start Date': 'first',
        'Contract End Date': 'last'
    }).reset_index()
    
    contract_info['contract_duration_days'] = (
        contract_info['Contract End Date'] - contract_info['Contract Start Date']
    ).dt.days
    
    features = features.merge(contract_info[['Unique ID', 'contract_duration_days']], on='Unique ID', how='left')
    
    # Lapse indicator
    features = features.merge(
        df.groupby('Unique ID')['Lapse Date'].first(),
        on='Unique ID'
    )
    features['has_lapsed'] = features['Lapse Date'].notna().astype(int)
    
    return features

demographic_features = create_demographic_features(df, observation_date)
print(f"\nFeatures created: {demographic_features.shape[1]}")
demographic_features.head()



Features created: 16


,Unique ID,Year of Birth,age,age_group,gender_Female,gender_Male,gender_Other,Original Date of Joining,membership_tenure_days,membership_tenure_years,Registration Status,is_active_member,Scheme Category/ Section Name,contract_duration_days,Lapse Date,has_lapsed
0,MEM00000001-01,1968,56,55-64,False,True,False,2018-09-05,2096,5.738535,Active,1,Advanced Cover,365,None,0
1,MEM00000001-02,1967,57,55-64,True,False,False,2019-05-27,1832,5.015743,Active,1,Executive Cover,365,None,0
2,MEM00000002-01,1976,48,45-54,True,False,False,2020-09-11,1359,3.720739,Active,1,Premier Cover,365,None,0
3,MEM00000002-03,2005,19,<25,False,True,False,2019-07-23,1775,4.859685,Active,1,Enhanced Plan,365,None,0
4,MEM00000003-01,1986,38,35-44,False,True,False,2024-04-25,37,0.101300,Active,1,Core Cover,365,None,0


## PHASE 2: MODEL TRAINING & EVALUATION


### STEP 5: Train-Test Split & Data Preparation


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

def prepare_modeling_data(df, selected_features, target_col='is_at_risk'):
    """
    Prepare final dataset for modeling
    """
    # Select features and target
    feature_cols = [col for col in selected_features if col in df.columns]
    
    # Handle any missing features
    available_features = [col for col in feature_cols if col in df.columns]
    print(f"Using {len(available_features)} features for modeling")
    
    X = df[available_features].copy()
    y = df[target_col]
    
    # Handle non-numeric columns
    for col in X.columns:
        if X[col].dtype == 'object':
            # Convert to category codes
            if X[col].nunique() < 50:
                X[col] = X[col].astype('category').cat.codes
            else:
                X[col] = pd.Categorical(X[col]).codes
        elif X[col].dtype == 'bool':
            X[col] = X[col].astype(int)
    
    # Fill any remaining NaN
    X = X.fillna(0)
    
    # Train-test split (80-20)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        random_state=42, 
        stratify=y
    )
    
    print(f"\nTraining set: {X_train.shape[0]} samples")
    print(f"Test set: {X_test.shape[0]} samples")
    print(f"\nTarget distribution (Train):")
    print(y_train.value_counts(normalize=True))
    print(f"\nTarget distribution (Test):")
    print(y_test.value_counts(normalize=True))
    
    return X_train, X_test, y_train, y_test, available_features

# Prepare data
X_train, X_test, y_train, y_test, final_features = prepare_modeling_data(
    processed_df, 
    selected_features, 
    target_col='is_at_risk'
)


### STEP 6: Handle Class Imbalance


In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

def balance_training_data(X_train, y_train, method='smote'):
    """
    Balance the training data to handle class imbalance
    """
    print(f"Original class distribution:")
    print(y_train.value_counts())
    print(f"\nAt-risk rate: {y_train.mean()*100:.2f}%")
    
    if method == 'smote':
        # SMOTE: Synthetic Minority Oversampling Technique
        smote = SMOTE(random_state=42, k_neighbors=5)
        X_balanced, y_balanced = smote.fit_resample(X_train, y_train)
        
    elif method == 'combined':
        # Combine SMOTE with undersampling
        over = SMOTE(sampling_strategy=0.1, random_state=42)
        under = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
        pipeline = ImbPipeline(steps=[('over', over), ('under', under)])
        X_balanced, y_balanced = pipeline.fit_resample(X_train, y_train)
    
    else:
        # No balancing
        X_balanced, y_balanced = X_train.copy(), y_train.copy()
    
    print(f"\nBalanced class distribution:")
    print(pd.Series(y_balanced).value_counts())
    print(f"\nBalanced at-risk rate: {y_balanced.mean()*100:.2f}%")
    
    return X_balanced, y_balanced

# Balance training data
X_train_balanced, y_train_balanced = balance_training_data(X_train, y_train, method='smote')


### STEP 7: Train Multiple Models


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, average_precision_score,
    precision_recall_curve, roc_curve
)
import time

def train_baseline_models(X_train, y_train, X_test, y_test):
    """
    Train multiple baseline models for comparison
    """
    
    models = {
        'Logistic Regression': LogisticRegression(
            max_iter=1000,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1
        ),
        'Random Forest': RandomForestClassifier(
            n_estimators=200,
            max_depth=10,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1
        ),
        'Gradient Boosting': GradientBoostingClassifier(
            n_estimators=200,
            learning_rate=0.1,
            max_depth=5,
            random_state=42
        ),
        'XGBoost': XGBClassifier(
            n_estimators=200,
            learning_rate=0.1,
            max_depth=5,
            scale_pos_weight=len(y_train[y_train==0])/len(y_train[y_train==1]),
            random_state=42,
            eval_metric='logloss',
            n_jobs=-1
        ),
        'LightGBM': LGBMClassifier(
            n_estimators=200,
            learning_rate=0.1,
            max_depth=7,
            class_weight='balanced',
            random_state=42,
            verbose=-1,
            n_jobs=-1
        )
    }
    
    results = {}
    trained_models = {}
    
    print("=== Training Models ===\n")
    
    for name, model in models.items():
        print(f"Training {name}...")
        start_time = time.time()
        
        # Train
        model.fit(X_train, y_train)
        training_time = time.time() - start_time
        
        # Predict
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        # Evaluate
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        pr_auc = average_precision_score(y_test, y_pred_proba)
        
        # Classification report
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        
        results[name] = {
            'roc_auc': roc_auc,
            'pr_auc': pr_auc,
            'precision': report['1']['precision'],
            'recall': report['1']['recall'],
            'f1_score': report['1']['f1-score'],
            'training_time': training_time,
            'predictions': y_pred,
            'probabilities': y_pred_proba
        }
        
        trained_models[name] = model
        
        print(f"  ROC-AUC: {roc_auc:.4f}")
        print(f"  PR-AUC: {pr_auc:.4f}")
        print(f"  Precision: {report['1']['precision']:.4f}")
        print(f"  Recall: {report['1']['recall']:.4f}")
        print(f"  F1-Score: {report['1']['f1-score']:.4f}")
        print(f"  Training Time: {training_time:.2f}s\n")
    
    # Compare results
    comparison_df = pd.DataFrame({
        name: {
            'ROC-AUC': results[name]['roc_auc'],
            'PR-AUC': results[name]['pr_auc'],
            'Precision': results[name]['precision'],
            'Recall': results[name]['recall'],
            'F1-Score': results[name]['f1_score'],
            'Training Time (s)': results[name]['training_time']
        }
        for name in results
    }).T.sort_values('PR-AUC', ascending=False)
    
    print("=== Model Comparison ===")
    print(comparison_df)
    
    return trained_models, results, comparison_df

# Train all models
trained_models, model_results, comparison_df = train_baseline_models(
    X_train_balanced, y_train_balanced, X_test, y_test
)


### STEP 8: Hyperparameter Tuning (Best Model)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import randint, uniform

def tune_best_model(X_train, y_train, model_type='lightgbm'):
    """
    Perform hyperparameter tuning on the best performing model
    """
    
    if model_type == 'lightgbm':
        param_distributions = {
            'n_estimators': randint(100, 500),
            'learning_rate': uniform(0.01, 0.2),
            'max_depth': randint(3, 10),
            'num_leaves': randint(20, 100),
            'min_child_samples': randint(10, 50),
            'subsample': uniform(0.6, 0.4),
            'colsample_bytree': uniform(0.6, 0.4),
            'reg_alpha': uniform(0, 1),
            'reg_lambda': uniform(0, 1)
        }
        
        base_model = LGBMClassifier(
            class_weight='balanced',
            random_state=42,
            verbose=-1,
            n_jobs=-1
        )
    
    elif model_type == 'xgboost':
        param_distributions = {
            'n_estimators': randint(100, 500),
            'learning_rate': uniform(0.01, 0.2),
            'max_depth': randint(3, 10),
            'min_child_weight': randint(1, 10),
            'subsample': uniform(0.6, 0.4),
            'colsample_bytree': uniform(0.6, 0.4),
            'gamma': uniform(0, 0.5),
            'reg_alpha': uniform(0, 1),
            'reg_lambda': uniform(0, 1)
        }
        
        scale_pos_weight = len(y_train[y_train==0])/len(y_train[y_train==1])
        base_model = XGBClassifier(
            scale_pos_weight=scale_pos_weight,
            random_state=42,
            eval_metric='logloss',
            n_jobs=-1
        )
    
    print(f"=== Hyperparameter Tuning for {model_type.upper()} ===\n")
    
    # Stratified K-Fold for cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Randomized search
    random_search = RandomizedSearchCV(
        base_model,
        param_distributions=param_distributions,
        n_iter=50,  # number of parameter combinations to try
        scoring='roc_auc',
        cv=cv,
        random_state=42,
        n_jobs=-1,
        verbose=1
    )
    
    # Fit
    random_search.fit(X_train, y_train)
    
    print(f"\nBest parameters:")
    for param, value in random_search.best_params_.items():
        print(f"  {param}: {value}")
    
    print(f"\nBest CV Score (ROC-AUC): {random_search.best_score_:.4f}")
    
    return random_search.best_estimator_, random_search.best_params_

# Identify best model from comparison
best_model_name = comparison_df.index[0]
print(f"Best baseline model: {best_model_name}\n")

# Tune the best model (using LightGBM as default if it's best, otherwise XGBoost)
if 'LightGBM' in best_model_name or 'lightgbm' in best_model_name.lower():
    best_model, best_params = tune_best_model(X_train_balanced, y_train_balanced, model_type='lightgbm')
elif 'XGBoost' in best_model_name or 'xgboost' in best_model_name.lower():
    best_model, best_params = tune_best_model(X_train_balanced, y_train_balanced, model_type='xgboost')
else:
    # Use LightGBM as default for tuning
    print("Using LightGBM for hyperparameter tuning...")
    best_model, best_params = tune_best_model(X_train_balanced, y_train_balanced, model_type='lightgbm')


### STEP 9: Final Model Evaluation


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate_final_model(model, X_test, y_test, threshold=0.5):
    """
    Comprehensive evaluation of the final model
    """
    
    # Predictions
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba >= threshold).astype(int)
    
    print("=== FINAL MODEL EVALUATION ===\n")
    
    # === 1. CLASSIFICATION METRICS ===
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=['Not At-Risk', 'At-Risk']))
    
    # === 2. CONFUSION MATRIX ===
    cm = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(cm)
    
    tn, fp, fn, tp = cm.ravel()
    print(f"\nTrue Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    # === 3. ROC-AUC ===
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    print(f"\nROC-AUC Score: {roc_auc:.4f}")
    
    # === 4. PRECISION-RECALL AUC ===
    pr_auc = average_precision_score(y_test, y_pred_proba)
    print(f"Precision-Recall AUC: {pr_auc:.4f}")
    
    # === 5. BUSINESS METRICS ===
    # Capture rate in top deciles
    df_scores = pd.DataFrame({
        'actual': y_test,
        'predicted_proba': y_pred_proba
    }).sort_values('predicted_proba', ascending=False)
    
    total_positives = df_scores['actual'].sum()
    
    print("\n=== Capture Rate Analysis ===")
    for decile in [10, 20, 30]:
        n = int(len(df_scores) * (decile/100))
        top_decile = df_scores.head(n)
        captured = top_decile['actual'].sum()
        capture_rate = (captured / total_positives * 100) if total_positives > 0 else 0
        print(f"Top {decile}%: Captures {capture_rate:.1f}% of at-risk members ({captured}/{total_positives})")
    
    # === 6. VISUALIZATIONS ===
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Confusion Matrix Heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0, 0])
    axes[0, 0].set_title('Confusion Matrix')
    axes[0, 0].set_ylabel('Actual')
    axes[0, 0].set_xlabel('Predicted')
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    axes[0, 1].plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.4f})')
    axes[0, 1].plot([0, 1], [0, 1], 'k--', label='Random')
    axes[0, 1].set_xlabel('False Positive Rate')
    axes[0, 1].set_ylabel('True Positive Rate')
    axes[0, 1].set_title('ROC Curve')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    axes[1, 0].plot(recall, precision, label=f'PR Curve (AUC = {pr_auc:.4f})')
    axes[1, 0].set_xlabel('Recall')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].set_title('Precision-Recall Curve')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    # Prediction Distribution
    axes[1, 1].hist(y_pred_proba[y_test == 0], bins=50, alpha=0.5, label='Not At-Risk', density=True)
    axes[1, 1].hist(y_pred_proba[y_test == 1], bins=50, alpha=0.5, label='At-Risk', density=True)
    axes[1, 1].axvline(threshold, color='r', linestyle='--', label=f'Threshold = {threshold}')
    axes[1, 1].set_xlabel('Predicted Probability')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].set_title('Prediction Probability Distribution')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    return {
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc,
        'confusion_matrix': cm
    }

# Evaluate final model
final_evaluation = evaluate_final_model(best_model, X_test, y_test)


## PHASE 3: CAUSAL ANALYTICS & EXPLORATORY ANALYSIS


### STEP 10: Feature Importance Analysis


In [ ]:
def analyze_feature_importance(model, feature_names, top_n=30):
    """
    Analyze and visualize feature importance
    """
    
    # Get feature importance
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    elif hasattr(model, 'coef_'):
        # For linear models, use absolute coefficients
        importances = np.abs(model.coef_[0])
    else:
        print("Model does not support feature importance extraction")
        return None
    
    # Create importance dataframe
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False)
    
    print("=== Top Feature Importances ===\n")
    print(importance_df.head(top_n))
    
    # Visualization
    plt.figure(figsize=(12, 8))
    top_features = importance_df.head(top_n)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Importance')
    plt.title(f'Top {top_n} Most Important Features')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    return importance_df

# Analyze feature importance
feature_importance_df = analyze_feature_importance(best_model, final_features, top_n=30)


### STEP 11: Causal Analytics - Root Cause Analysis


In [ ]:
def causal_analytics(df, target_col='is_at_risk'):
    """
    Perform causal analytics to identify root causes of high risk
    """
    
    print("=== CAUSAL ANALYTICS: ROOT CAUSE ANALYSIS ===\n")
    
    # === 1. DEMOGRAPHIC FACTORS ===
    print("1. DEMOGRAPHIC FACTORS\n")
    
    # Age groups
    if 'age_group' in df.columns:
        age_risk = df.groupby('age_group')[target_col].agg(['mean', 'count'])
        age_risk.columns = ['At-Risk Rate', 'Count']
        age_risk = age_risk.sort_values('At-Risk Rate', ascending=False)
        print("At-Risk Rate by Age Group:")
        print(age_risk)
        print()
    
    # Gender
    gender_cols = [col for col in df.columns if 'gender_' in col.lower()]
    if gender_cols:
        for col in gender_cols:
            gender_name = col.replace('gender_', '').replace('_', ' ').title()
            risk_rate = df[df[col] == True][target_col].mean() if df[col].dtype == bool else df[df[col] == 1][target_col].mean()
            count = (df[col] == True).sum() if df[col].dtype == bool else (df[col] == 1).sum()
            print(f"{gender_name}: {risk_rate*100:.2f}% at-risk (n={count})")
        print()
    
    # === 2. CLAIMS HISTORY FACTORS ===
    print("2. CLAIMS HISTORY FACTORS\n")
    
    # Recent activity
    if 'claims_count_3m' in df.columns:
        df['has_recent_claims'] = (df['claims_count_3m'] > 0).astype(int)
        recent_risk = df.groupby('has_recent_claims')[target_col].agg(['mean', 'count'])
        recent_risk.index = ['No Recent Claims', 'Has Recent Claims']
        recent_risk.columns = ['At-Risk Rate', 'Count']
        print("At-Risk Rate by Recent Claims (3 months):")
        print(recent_risk)
        print()
    
    # High cost claims
    if 'high_cost_claims_count' in df.columns:
        df['has_high_cost'] = (df['high_cost_claims_count'] > 0).astype(int)
        high_cost_risk = df.groupby('has_high_cost')[target_col].agg(['mean', 'count'])
        high_cost_risk.index = ['No High-Cost Claims', 'Has High-Cost Claims']
        high_cost_risk.columns = ['At-Risk Rate', 'Count']
        print("At-Risk Rate by High-Cost Claims:")
        print(high_cost_risk)
        print()
    
    # === 3. CONDITION FACTORS ===
    print("3. CONDITION FACTORS\n")
    
    if 'has_high_risk_condition_history' in df.columns:
        condition_risk = df.groupby('has_high_risk_condition_history')[target_col].agg(['mean', 'count'])
        condition_risk.index = ['No High-Risk Conditions', 'Has High-Risk Conditions']
        condition_risk.columns = ['At-Risk Rate', 'Count']
        print("At-Risk Rate by High-Risk Condition History:")
        print(condition_risk)
        print()
    
    # === 4. MEMBERSHIP FACTORS ===
    print("4. MEMBERSHIP FACTORS\n")
    
    if 'membership_tenure_years' in df.columns:
        df['tenure_group'] = pd.cut(
            df['membership_tenure_years'],
            bins=[0, 1, 3, 5, 10, 100],
            labels=['<1 year', '1-3 years', '3-5 years', '5-10 years', '10+ years']
        )
        tenure_risk = df.groupby('tenure_group')[target_col].agg(['mean', 'count'])
        tenure_risk.columns = ['At-Risk Rate', 'Count']
        print("At-Risk Rate by Membership Tenure:")
        print(tenure_risk)
        print()
    
    # === 5. COMPOSITE RISK SCORE ANALYSIS ===
    print("5. COMPOSITE RISK SCORE ANALYSIS\n")
    
    if 'composite_risk_score' in df.columns:
        df['risk_quartile'] = pd.qcut(
            df['composite_risk_score'],
            q=4,
            labels=['Q1 (Lowest)', 'Q2', 'Q3', 'Q4 (Highest)']
        )
        quartile_risk = df.groupby('risk_quartile')[target_col].agg(['mean', 'count'])
        quartile_risk.columns = ['At-Risk Rate', 'Count']
        print("At-Risk Rate by Risk Score Quartile:")
        print(quartile_risk)
        print()
    
    # === 6. CORRELATION ANALYSIS ===
    print("6. TOP CORRELATIONS WITH AT-RISK STATUS\n")
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    correlations = df[numeric_cols].corrwith(df[target_col]).abs().sort_values(ascending=False)
    top_correlations = correlations.head(15)
    print(top_correlations)
    
    return {
        'age_risk': age_risk if 'age_group' in df.columns else None,
        'recent_claims_risk': recent_risk if 'claims_count_3m' in df.columns else None,
        'high_cost_risk': high_cost_risk if 'high_cost_claims_count' in df.columns else None,
        'condition_risk': condition_risk if 'has_high_risk_condition_history' in df.columns else None,
        'tenure_risk': tenure_risk if 'membership_tenure_years' in df.columns else None,
        'correlations': top_correlations
    }

# Perform causal analytics
causal_results = causal_analytics(processed_df, target_col='is_at_risk')


### STEP 12: Cohort Analysis


In [ ]:
def cohort_analysis(df, target_col='is_at_risk'):
    """
    Identify high-risk cohorts for targeted interventions
    """
    
    print("=== COHORT ANALYSIS: HIGH-RISK GROUPS ===\n")
    
    # Create cohort definitions
    cohorts = []
    
    # Cohort 1: Recent High Activity + High Cost
    if all(col in df.columns for col in ['claims_count_3m', 'high_cost_claims_count']):
        cohort1 = df[
            (df['claims_count_3m'] >= 2) & 
            (df['high_cost_claims_count'] > 0)
        ].copy()
        if len(cohort1) > 0:
            cohorts.append({
                'name': 'Recent High Activity + High Cost',
                'criteria': '≥2 claims in 3m AND has high-cost claims',
                'size': len(cohort1),
                'at_risk_rate': cohort1[target_col].mean(),
                'avg_risk_score': cohort1['composite_risk_score'].mean() if 'composite_risk_score' in cohort1.columns else None,
                'members': cohort1
            })
    
    # Cohort 2: High-Risk Condition History
    if 'has_high_risk_condition_history' in df.columns:
        cohort2 = df[df['has_high_risk_condition_history'] == 1].copy()
        if len(cohort2) > 0:
            cohorts.append({
                'name': 'High-Risk Condition History',
                'criteria': 'Has high-risk condition in history',
                'size': len(cohort2),
                'at_risk_rate': cohort2[target_col].mean(),
                'avg_risk_score': cohort2['composite_risk_score'].mean() if 'composite_risk_score' in cohort2.columns else None,
                'members': cohort2
            })
    
    # Cohort 3: Accelerating Claims
    if all(col in df.columns for col in ['claims_count_6m', 'claims_count_24m']):
        df['claim_acceleration'] = (df['claims_count_6m'] / 6) / ((df['claims_count_24m'] - df['claims_count_6m']) / 18 + 0.1)
        cohort3 = df[df['claim_acceleration'] > 2].copy()
        if len(cohort3) > 0:
            cohorts.append({
                'name': 'Accelerating Claims',
                'criteria': 'Claim frequency accelerating (recent > 2x historical)',
                'size': len(cohort3),
                'at_risk_rate': cohort3[target_col].mean(),
                'avg_risk_score': cohort3['composite_risk_score'].mean() if 'composite_risk_score' in cohort3.columns else None,
                'members': cohort3
            })
    
    # Cohort 4: High Composite Risk Score
    if 'composite_risk_score' in df.columns:
        risk_threshold = df['composite_risk_score'].quantile(0.90)
        cohort4 = df[df['composite_risk_score'] >= risk_threshold].copy()
        if len(cohort4) > 0:
            cohorts.append({
                'name': 'Top 10% Risk Score',
                'criteria': f'Composite risk score ≥ {risk_threshold:.2f} (90th percentile)',
                'size': len(cohort4),
                'at_risk_rate': cohort4[target_col].mean(),
                'avg_risk_score': cohort4['composite_risk_score'].mean(),
                'members': cohort4
            })
    
    # Cohort 5: Age + Condition Combination
    if all(col in df.columns for col in ['age', 'has_high_risk_condition_history']):
        cohort5 = df[
            (df['age'] >= 55) & 
            (df['has_high_risk_condition_history'] == 1)
        ].copy()
        if len(cohort5) > 0:
            cohorts.append({
                'name': 'Older Adults with High-Risk Conditions',
                'criteria': 'Age ≥55 AND has high-risk condition history',
                'size': len(cohort5),
                'at_risk_rate': cohort5[target_col].mean(),
                'avg_risk_score': cohort5['composite_risk_score'].mean() if 'composite_risk_score' in cohort5.columns else None,
                'members': cohort5
            })
    
    # Display cohort summary
    cohort_summary = pd.DataFrame({
        'Cohort': [c['name'] for c in cohorts],
        'Size': [c['size'] for c in cohorts],
        'At-Risk Rate': [f"{c['at_risk_rate']*100:.2f}%" for c in cohorts],
        'Avg Risk Score': [f"{c['avg_risk_score']:.2f}" if c['avg_risk_score'] else "N/A" for c in cohorts]
    })
    
    print("Cohort Summary:")
    print(cohort_summary)
    print()
    
    # Visualize cohort comparison
    if len(cohorts) > 0:
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # At-risk rate by cohort
        cohort_names = [c['name'] for c in cohorts]
        at_risk_rates = [c['at_risk_rate']*100 for c in cohorts]
        
        axes[0].barh(cohort_names, at_risk_rates)
        axes[0].set_xlabel('At-Risk Rate (%)')
        axes[0].set_title('At-Risk Rate by Cohort')
        axes[0].grid(True, axis='x')
        
        # Cohort sizes
        cohort_sizes = [c['size'] for c in cohorts]
        axes[1].barh(cohort_names, cohort_sizes)
        axes[1].set_xlabel('Number of Members')
        axes[1].set_title('Cohort Sizes')
        axes[1].grid(True, axis='x')
        
        plt.tight_layout()
        plt.show()
    
    return cohorts

# Perform cohort analysis
high_risk_cohorts = cohort_analysis(processed_df, target_col='is_at_risk')


### STEP 13: Intervention Recommendations


In [ ]:
def generate_intervention_recommendations(cohorts, feature_importance_df, causal_results):
    """
    Generate actionable intervention recommendations based on analysis
    """
    
    print("=== INTERVENTION RECOMMENDATIONS ===\n")
    
    recommendations = []
    
    # === 1. BASED ON TOP FEATURES ===
    print("1. INTERVENTIONS BASED ON KEY RISK FACTORS\n")
    
    if feature_importance_df is not None:
        top_features = feature_importance_df.head(10)
        
        for idx, row in top_features.iterrows():
            feature = row['feature']
            importance = row['importance']
            
            # Map features to interventions
            if 'recent_activity' in feature.lower() or 'claims_count_3m' in feature.lower():
                recommendations.append({
                    'priority': 'HIGH',
                    'target_group': 'Members with recent high claim activity',
                    'intervention': 'Proactive health check-ups and wellness consultations',
                    'rationale': f'Recent activity is a strong predictor (importance: {importance:.4f})',
                    'expected_impact': 'Early detection and prevention of complications'
                })
            
            elif 'high_risk_condition' in feature.lower() or 'chronic' in feature.lower():
                recommendations.append({
                    'priority': 'HIGH',
                    'target_group': 'Members with high-risk condition history',
                    'intervention': 'Disease management programs and regular monitoring',
                    'rationale': f'Condition history is critical (importance: {importance:.4f})',
                    'expected_impact': 'Prevent condition progression and complications'
                })
            
            elif 'composite_risk_score' in feature.lower():
                recommendations.append({
                    'priority': 'MEDIUM',
                    'target_group': 'Members in top risk quartiles',
                    'intervention': 'Comprehensive risk assessment and personalized care plans',
                    'rationale': f'Composite score captures multiple risk factors (importance: {importance:.4f})',
                    'expected_impact': 'Holistic risk management'
                })
            
            elif 'membership_tenure' in feature.lower():
                recommendations.append({
                    'priority': 'MEDIUM',
                    'target_group': 'New members (<1 year)',
                    'intervention': 'Enhanced onboarding and health screening',
                    'rationale': f'Membership tenure affects risk patterns (importance: {importance:.4f})',
                    'expected_impact': 'Early identification and engagement'
                })
    
    # === 2. BASED ON COHORTS ===
    print("\n2. COHORT-SPECIFIC INTERVENTIONS\n")
    
    for cohort in cohorts:
        cohort_name = cohort['name']
        at_risk_rate = cohort['at_risk_rate']
        
        if 'Recent High Activity' in cohort_name:
            recommendations.append({
                'priority': 'HIGH',
                'target_group': cohort_name,
                'intervention': 'Immediate case management and care coordination',
                'rationale': f'{at_risk_rate*100:.1f}% at-risk rate in this cohort',
                'expected_impact': 'Prevent escalation of claims and conditions'
            })
        
        elif 'High-Risk Condition' in cohort_name:
            recommendations.append({
                'priority': 'HIGH',
                'target_group': cohort_name,
                'intervention': 'Specialized disease management and specialist referrals',
                'rationale': f'{at_risk_rate*100:.1f}% at-risk rate - condition-specific care needed',
                'expected_impact': 'Better disease control and reduced complications'
            })
        
        elif 'Accelerating' in cohort_name:
            recommendations.append({
                'priority': 'HIGH',
                'target_group': cohort_name,
                'intervention': 'Urgent health assessment and intervention planning',
                'rationale': f'{at_risk_rate*100:.1f}% at-risk rate - claims accelerating',
                'expected_impact': 'Arrest the acceleration trend'
            })
        
        elif 'Older Adults' in cohort_name:
            recommendations.append({
                'priority': 'MEDIUM',
                'target_group': cohort_name,
                'intervention': 'Geriatric care management and preventive screenings',
                'rationale': f'{at_risk_rate*100:.1f}% at-risk rate - age-related risk',
                'expected_impact': 'Age-appropriate care and prevention'
            })
    
    # === 3. GENERAL RECOMMENDATIONS ===
    print("\n3. GENERAL RECOMMENDATIONS\n")
    
    general_recommendations = [
        {
            'priority': 'MEDIUM',
            'target_group': 'All members with risk score > 50th percentile',
            'intervention': 'Regular health assessments and wellness programs',
            'rationale': 'Preventive care for moderate-risk members',
            'expected_impact': 'Early intervention and risk reduction'
        },
        {
            'priority': 'LOW',
            'target_group': 'Low-risk members',
            'intervention': 'Maintain wellness programs and preventive care',
            'rationale': 'Keep low-risk members healthy',
            'expected_impact': 'Prevent risk escalation'
        }
    ]
    
    recommendations.extend(general_recommendations)
    
    # === 4. BENEFIT DESIGN RECOMMENDATIONS ===
    print("\n4. BENEFIT DESIGN RECOMMENDATIONS\n")
    
    benefit_recommendations = [
        {
            'recommendation': 'Enhanced preventive care coverage',
            'rationale': 'Early detection reduces high-cost claims',
            'target': 'All members, especially high-risk cohorts'
        },
        {
            'recommendation': 'Disease management program incentives',
            'rationale': 'Encourage participation in chronic disease management',
            'target': 'Members with high-risk conditions'
        },
        {
            'recommendation': 'Wellness program rewards',
            'rationale': 'Incentivize healthy behaviors',
            'target': 'Moderate-risk members'
        },
        {
            'recommendation': 'Care coordination services',
            'rationale': 'Reduce fragmented care and improve outcomes',
            'target': 'High-risk members with multiple conditions'
        }
    ]
    
    # Display recommendations
    rec_df = pd.DataFrame(recommendations)
    
    print("Intervention Recommendations:")
    print(rec_df.to_string(index=False))
    print()
    
    print("Benefit Design Recommendations:")
    for rec in benefit_recommendations:
        print(f"\n• {rec['recommendation']}")
        print(f"  Rationale: {rec['rationale']}")
        print(f"  Target: {rec['target']}")
    
    # Priority summary
    print("\n=== PRIORITY SUMMARY ===\n")
    priority_summary = rec_df.groupby('priority').size()
    print(priority_summary)
    
    return recommendations, benefit_recommendations

# Generate recommendations
interventions, benefit_design = generate_intervention_recommendations(
    high_risk_cohorts, 
    feature_importance_df, 
    causal_results
)


### STEP 14: Model Deployment Preparation


In [ ]:
import joblib
import json
from datetime import datetime

def save_model_and_artifacts(model, feature_names, model_results, best_params):
    """
    Save the trained model and all necessary artifacts for deployment
    """
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save model
    model_filename = f'at_risk_model_{timestamp}.pkl'
    joblib.dump(model, model_filename)
    print(f"✅ Model saved: {model_filename}")
    
    # Save feature names
    features_filename = f'model_features_{timestamp}.json'
    with open(features_filename, 'w') as f:
        json.dump(feature_names, f)
    print(f"✅ Features saved: {features_filename}")
    
    # Save model metadata
    metadata = {
        'model_type': type(model).__name__,
        'training_date': timestamp,
        'features': feature_names,
        'n_features': len(feature_names),
        'best_parameters': best_params,
        'performance': {
            'roc_auc': float(model_results.get('roc_auc', 0)),
            'pr_auc': float(model_results.get('pr_auc', 0)),
            'precision': float(model_results.get('precision', 0)),
            'recall': float(model_results.get('recall', 0)),
            'f1_score': float(model_results.get('f1_score', 0))
        }
    }
    
    metadata_filename = f'model_metadata_{timestamp}.json'
    with open(metadata_filename, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"✅ Metadata saved: {metadata_filename}")
    
    return model_filename, features_filename, metadata_filename

# Save model artifacts
model_file, features_file, metadata_file = save_model_and_artifacts(
    best_model,
    final_features,
    model_results.get(best_model_name, {}),
    best_params
)

print("\n=== MODEL DEPLOYMENT READY ===")
print(f"Model: {model_file}")
print(f"Features: {features_file}")
print(f"Metadata: {metadata_file}")


### STEP 15: Prediction Function for New Members


In [ ]:
def predict_at_risk(member_data, model, feature_names, threshold=0.5):
    """
    Predict at-risk status for a new member or set of members
    
    Parameters:
    - member_data: DataFrame with member features
    - model: Trained model
    - feature_names: List of feature names used in training
    - threshold: Probability threshold for classification
    
    Returns:
    - DataFrame with predictions and probabilities
    """
    
    # Select and align features
    X = member_data[feature_names].copy()
    
    # Handle missing features (fill with 0)
    for feature in feature_names:
        if feature not in X.columns:
            X[feature] = 0
    
    # Ensure correct order
    X = X[feature_names]
    
    # Handle non-numeric columns
    for col in X.columns:
        if X[col].dtype == 'object':
            if X[col].nunique() < 50:
                X[col] = X[col].astype('category').cat.codes
            else:
                X[col] = pd.Categorical(X[col]).codes
        elif X[col].dtype == 'bool':
            X[col] = X[col].astype(int)
    
    # Fill NaN
    X = X.fillna(0)
    
    # Predict
    probabilities = model.predict_proba(X)[:, 1]
    predictions = (probabilities >= threshold).astype(int)
    
    # Create results dataframe
    results = pd.DataFrame({
        'member_id': member_data.index if 'Unique ID' not in member_data.columns else member_data['Unique ID'],
        'at_risk_probability': probabilities,
        'is_at_risk': predictions,
        'risk_level': pd.cut(
            probabilities,
            bins=[0, 0.3, 0.6, 1.0],
            labels=['Low', 'Medium', 'High']
        )
    })
    
    return results

# Example: Predict on test set
print("=== EXAMPLE PREDICTIONS ===")
test_predictions = predict_at_risk(
    processed_df.loc[X_test.index].copy(),
    best_model,
    final_features,
    threshold=0.5
)

print(f"\nPrediction Summary:")
print(test_predictions['risk_level'].value_counts())
print(f"\nSample Predictions:")
print(test_predictions.head(10))


## SUMMARY & NEXT STEPS

### Project Completion Summary

✅ **Completed:**
1. Data preparation and feature engineering (58 features)
2. Model training with 5 algorithms
3. Hyperparameter tuning for best model
4. Comprehensive model evaluation
5. Feature importance analysis
6. Causal analytics and root cause identification
7. Cohort analysis for high-risk groups
8. Intervention recommendations
9. Model deployment preparation

### Key Findings

1. **Best Model**: [Check comparison_df for best performing model]
2. **Top Risk Factors**: [Check feature_importance_df]
3. **High-Risk Cohorts**: [Check high_risk_cohorts]
4. **Intervention Priorities**: [Check interventions]

### Next Steps

1. **Model Monitoring**: Set up monitoring for model performance over time
2. **A/B Testing**: Test interventions on identified high-risk cohorts
3. **Feedback Loop**: Collect outcomes data to improve model
4. **Production Deployment**: Deploy model to production environment
5. **Stakeholder Reporting**: Create dashboards for business users


#### 3.2 Historical Claims Features (CRITICAL)

In [ ]:
def create_claims_history_features(claims_df, observation_date):
    """
    Create comprehensive historical claims features
    """
    
    observation_date = pd.to_datetime(observation_date)
    claims_df['Paid Date'] = pd.to_datetime(claims_df['Paid Date'])
    
    # Calculate time periods
    claims_df['months_before_observation'] = (
        (observation_date.year - claims_df['Paid Date'].dt.year) * 12 +
        (observation_date.month - claims_df['Paid Date'].dt.month)
    )
    
    features_list = []
    
    # === 1. BASIC CLAIM COUNTS & AMOUNTS ===
    basic_agg = claims_df.groupby('Claimant Unique ID').agg({
        'Claim ID': 'count',  # total claims
        'Claim Amount': ['sum', 'mean', 'median', 'std', 'min', 'max'],
    }).reset_index()
    
    basic_agg.columns = ['Unique ID', 'total_claims_count', 'total_claim_amount', 
                         'avg_claim_amount', 'median_claim_amount', 'std_claim_amount',
                         'min_claim_amount', 'max_claim_amount']
    
    features_list.append(basic_agg)
    
    # === 2. TIME-WINDOWED FEATURES ===
    time_windows = [3, 6, 12, 24]  # months
    
    for window in time_windows:
        window_claims = claims_df[claims_df['months_before_observation'] <= window]
        
        window_agg = window_claims.groupby('Claimant Unique ID').agg({
            'Claim ID': 'count',
            'Claim Amount': 'sum'
        }).reset_index()
        
        window_agg.columns = ['Unique ID', f'claims_count_{window}m', f'total_amount_{window}m']
        
        features_list.append(window_agg)
    
    # === 3. TEMPORAL PATTERNS ===
    temporal = claims_df.groupby('Claimant Unique ID').agg({
        'Paid Date': ['min', 'max', 'count']
    }).reset_index()
    
    temporal.columns = ['Unique ID', 'first_claim_date', 'last_claim_date', 'claim_count']
    temporal['days_since_first_claim'] = (observation_date - temporal['first_claim_date']).dt.days
    temporal['days_since_last_claim'] = (observation_date - temporal['last_claim_date']).dt.days
    temporal['claim_duration_days'] = (temporal['last_claim_date'] - temporal['first_claim_date']).dt.days
    temporal['avg_days_between_claims'] = temporal['claim_duration_days'] / (temporal['claim_count'] - 1)
    temporal['avg_days_between_claims'] = temporal['avg_days_between_claims'].fillna(0)
    
    features_list.append(temporal[['Unique ID', 'days_since_first_claim', 'days_since_last_claim', 
                                   'claim_duration_days', 'avg_days_between_claims']])
    
    # === 4. CLAIM FREQUENCY TRENDS ===
    # Claims per month over time
    claims_df['claim_year_month'] = claims_df['Paid Date'].dt.to_period('M')
    monthly_claims = claims_df.groupby(['Claimant Unique ID', 'claim_year_month']).size().reset_index(name='monthly_count')
    
    freq_features = monthly_claims.groupby('Claimant Unique ID').agg({
        'monthly_count': ['mean', 'std', 'max']
    }).reset_index()
    
    freq_features.columns = ['Unique ID', 'avg_monthly_claim_frequency', 
                            'std_monthly_claim_frequency', 'max_monthly_claims']
    
    features_list.append(freq_features)
    
    # === 5. CLAIM SEVERITY INDICATORS ===
    # High-cost claims
    high_cost_threshold = claims_df['Claim Amount'].quantile(0.75)
    claims_df['is_high_cost_claim'] = (claims_df['Claim Amount'] > high_cost_threshold).astype(int)
    
    severity = claims_df.groupby('Claimant Unique ID').agg({
        'is_high_cost_claim': 'sum'
    }).reset_index()
    
    severity.columns = ['Unique ID', 'high_cost_claims_count']
    
    # Cost volatility
    cost_volatility = claims_df.groupby('Claimant Unique ID')['Claim Amount'].apply(
        lambda x: x.std() / x.mean() if x.mean() > 0 else 0
    ).reset_index()
    cost_volatility.columns = ['Unique ID', 'claim_amount_cv']  # coefficient of variation
    
    features_list.append(severity)
    features_list.append(cost_volatility)
    
    # === 6. TREND FEATURES (Increasing/Decreasing Claims) ===
    def calculate_trend(group):
        if len(group) < 2:
            return 0
        group = group.sort_values('Paid Date')
        group['time_index'] = range(len(group))
        correlation = group['time_index'].corr(group['Claim Amount'])
        return correlation if not np.isnan(correlation) else 0
    
    trends = claims_df.groupby('Claimant Unique ID').apply(calculate_trend).reset_index()
    trends.columns = ['Unique ID', 'claim_amount_trend']
    
    features_list.append(trends)
    
    # Merge all features
    from functools import reduce
    claims_features = reduce(lambda left, right: left.merge(right, on='Unique ID', how='outer'), features_list)
    
    # Fill NaN with 0 for members with no claims
    claims_features = claims_features.fillna(0)
    
    return claims_features

claims_features = create_claims_history_features(historical_claims, observation_date)
claims_features.describe()
print(f"Claims features created: {claims_features.shape[1]}")


C:\Temp\ipykernel_15380\29807541.py:102: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trends = claims_df.groupby('Claimant Unique ID').apply(calculate_trend).reset_index()


Claims features created: 26


In [32]:
claims_features.describe()

,total_claims_count,total_claim_amount,avg_claim_amount,median_claim_amount,std_claim_amount,min_claim_amount,max_claim_amount,claims_count_3m,total_amount_3m,claims_count_6m,...,days_since_first_claim,days_since_last_claim,claim_duration_days,avg_days_between_claims,avg_monthly_claim_frequency,std_monthly_claim_frequency,max_monthly_claims,high_cost_claims_count,claim_amount_cv,claim_amount_trend
count,32857.000000,3.285700e+04,32857.000000,32857.000000,32857.000000,32857.000000,32857.000000,32857.000000,3.285700e+04,32857.000000,...,32857.000000,32857.000000,32857.00000,32857.000000,32857.000000,32857.000000,32857.000000,32857.000000,32857.000000,32857.000000
mean,4.506011,2.558287e+04,5477.329289,4278.385454,4080.006793,3116.627819,13236.995414,0.320541,1.818936e+03,0.620659,...,713.900387,623.257997,90.64239,38.289121,1.227895,0.176047,1.480659,1.126518,0.535784,-0.002648
std,35.890613,2.142705e+05,16247.585935,15727.285264,12673.123009,14643.876577,33658.180062,4.926635,2.820913e+04,8.206342,...,418.583053,407.687561,122.78210,61.603699,2.541327,1.200852,3.767306,9.307321,0.753973,0.512540
min,1.000000,4.559000e+01,45.590000,45.590000,0.000000,40.660000,45.590000,0.000000,0.000000e+00,0.000000,...,1.000000,1.000000,0.00000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,-1.000000
25%,1.000000,6.366600e+02,508.520000,461.710000,0.000000,232.240000,571.400000,0.000000,0.000000e+00,0.000000,...,354.000000,257.000000,0.00000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,1.000000,1.809810e+03,1061.190000,814.615000,0.000000,451.340000,1386.110000,0.000000,0.000000e+00,0.000000,...,707.000000,606.000000,0.00000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,1.015042e+04,3547.030000,1660.740000,1338.369289,1013.670000,7696.810000,0.000000,0.000000e+00,0.000000,...,1064.000000,966.000000,182.00000,59.000000,1.000000,0.000000,1.000000,1.000000,0.987087,0.000000
max,3974.000000,2.349523e+07,249923.290000,249923.290000,175583.926365,249923.290000,286107.430000,574.000000,2.905958e+06,690.000000,...,1591.000000,1578.000000,440.00000,412.000000,264.933333,111.126355,359.000000,1018.000000,5.341215,1.000000


#### 3.3 Treatment & Condition Features

In [33]:
def create_treatment_condition_features(claims_df):
    """
    Create features from Treatment Type and Condition information
    """
    
    features_list = []
    
    # === 1. TREATMENT TYPE FEATURES ===
    treatment_features = claims_df.groupby('Claimant Unique ID').agg({
        'Treatment Type': lambda x: x.nunique()  # unique treatment types
    }).reset_index()
    treatment_features.columns = ['Unique ID', 'unique_treatment_types']
    
    # Most frequent treatment
    most_frequent_treatment = claims_df.groupby('Claimant Unique ID')['Treatment Type'].agg(
        lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown'
    ).reset_index()
    most_frequent_treatment.columns = ['Unique ID', 'most_frequent_treatment']
    
    # Treatment diversity (entropy)
    def calculate_entropy(series):
        from scipy.stats import entropy
        value_counts = series.value_counts(normalize=True)
        return entropy(value_counts) if len(value_counts) > 1 else 0
    
    treatment_diversity = claims_df.groupby('Claimant Unique ID')['Treatment Type'].apply(
        calculate_entropy
    ).reset_index()
    treatment_diversity.columns = ['Unique ID', 'treatment_diversity_entropy']
    
    features_list.extend([treatment_features, most_frequent_treatment, treatment_diversity])
    
    # === 2. CONDITION FEATURES ===
    condition_features = claims_df.groupby('Claimant Unique ID').agg({
        'Condition Code': 'nunique',
        'Condition Category': 'nunique'
    }).reset_index()
    condition_features.columns = ['Unique ID', 'unique_condition_codes', 'unique_condition_categories']
    
    # Check for chronic conditions (repeat condition codes)
    repeat_conditions = claims_df.groupby(['Claimant Unique ID', 'Condition Code']).size().reset_index(name='count')
    chronic_indicator = repeat_conditions[repeat_conditions['count'] >= 3].groupby('Claimant Unique ID').size().reset_index()
    chronic_indicator.columns = ['Unique ID', 'chronic_conditions_count']
    
    features_list.extend([condition_features, chronic_indicator])
    
    # === 3. HIGH-RISK CONDITION FLAGS ===
    high_risk_conditions = ['CANCER', 'CARDIOVASCULAR', 'DIABETES', 'CHRONIC KIDNEY', 'MENTAL HEALTH']
    
    def has_high_risk_condition(condition_series):
        condition_text = ' '.join(condition_series.astype(str).str.upper())
        return any(condition in condition_text for condition in high_risk_conditions)
    
    high_risk_flags = claims_df.groupby('Claimant Unique ID')['Condition Category'].apply(
        has_high_risk_condition
    ).reset_index()
    high_risk_flags.columns = ['Unique ID', 'has_high_risk_condition_history']
    high_risk_flags['has_high_risk_condition_history'] = high_risk_flags['has_high_risk_condition_history'].astype(int)
    
    features_list.append(high_risk_flags)
    
    # === 4. TREATMENT LOCATION FEATURES ===
    location_features = claims_df.groupby('Claimant Unique ID').agg({
        'Treatment Location': lambda x: x.nunique()
    }).reset_index()
    location_features.columns = ['Unique ID', 'unique_treatment_locations']
    
    # Inpatient vs Outpatient ratio (if available)
    # This depends on data structure
    
    features_list.append(location_features)
    
    # === 5. PROVIDER DIVERSITY ===
    if 'Provider Type' in claims_df.columns:
        provider_features = claims_df.groupby('Claimant Unique ID').agg({
            'Provider Type': 'nunique'
        }).reset_index()
        provider_features.columns = ['Unique ID', 'unique_providers']
        features_list.append(provider_features)
    
    # Merge all
    from functools import reduce
    treatment_condition_features = reduce(
        lambda left, right: left.merge(right, on='Unique ID', how='outer'), 
        features_list
    )
    
    treatment_condition_features = treatment_condition_features.fillna(0)
    
    return treatment_condition_features

treatment_condition_features = create_treatment_condition_features(historical_claims)
print(f"Treatment/Condition features: {treatment_condition_features.shape[1]}")

Treatment/Condition features: 10


#### 3.4 Risk Score Features (Composite Indicators)

In [53]:
treatment_condition_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32857 entries, 0 to 32856
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unique ID                        32857 non-null  object 
 1   unique_treatment_types           32857 non-null  int64  
 2   most_frequent_treatment          32857 non-null  object 
 3   treatment_diversity_entropy      32857 non-null  float64
 4   unique_condition_codes           32857 non-null  int64  
 5   unique_condition_categories      32857 non-null  int64  
 6   chronic_conditions_count         32857 non-null  float64
 7   has_high_risk_condition_history  32857 non-null  int32  
 8   unique_treatment_locations       32857 non-null  int64  
 9   unique_providers                 32857 non-null  int64  
dtypes: float64(2), int32(1), int64(5), object(2)
memory usage: 2.4+ MB


In [52]:
claims_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32857 entries, 0 to 32856
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unique ID                    32857 non-null  object 
 1   total_claims_count           32857 non-null  int64  
 2   total_claim_amount           32857 non-null  float64
 3   avg_claim_amount             32857 non-null  float64
 4   median_claim_amount          32857 non-null  float64
 5   std_claim_amount             32857 non-null  float64
 6   min_claim_amount             32857 non-null  float64
 7   max_claim_amount             32857 non-null  float64
 8   claims_count_3m              32857 non-null  float64
 9   total_amount_3m              32857 non-null  float64
 10  claims_count_6m              32857 non-null  float64
 11  total_amount_6m              32857 non-null  float64
 12  claims_count_12m             32857 non-null  float64
 13  total_amount_12m

In [43]:
demographic_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 16 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Unique ID                      120000 non-null  object        
 1   Year of Birth                  120000 non-null  int64         
 2   age                            120000 non-null  int64         
 3   age_group                      120000 non-null  category      
 4   gender_Female                  120000 non-null  bool          
 5   gender_Male                    120000 non-null  bool          
 6   gender_Other                   120000 non-null  bool          
 7   Original Date of Joining       120000 non-null  datetime64[ns]
 8   membership_tenure_days         120000 non-null  int64         
 9   membership_tenure_years        120000 non-null  float64       
 10  Registration Status            120000 non-null  object        
 11  

In [54]:
def create_risk_score_features(claims_features, demographic_features):
    """
    Create composite risk scores based on multiple factors
    """
    
    # Merge claims and demographic
    merged = demographic_features.merge(claims_features, on='Unique ID', how='left')
    merged = merged.merge(treatment_condition_features, on='Unique ID', how='left')

    # Handle NaNs by dtype
    for col in merged.columns:
        dtype = merged[col].dtype

        # Numeric columns
        if pd.api.types.is_numeric_dtype(dtype):
            merged[col] = merged[col].fillna(0)

        # Boolean columns
        elif pd.api.types.is_bool_dtype(dtype):
            merged[col] = merged[col].fillna(False)

        # Categorical columns
        elif pd.api.types.is_categorical_dtype(dtype):
            merged[col] = merged[col].cat.add_categories(['Unknown']).fillna('Unknown')

        # Object columns (string, text)
        elif pd.api.types.is_object_dtype(dtype):
            merged[col] = merged[col].fillna('Unknown')

        # Anything else — leave untouched
        else:
            pass

    df = merged.copy()

    # === 1. UTILIZATION RISK SCORE ===
    # Normalized by age and membership tenure
    df['claims_per_year'] = df['total_claims_count'] / (df['membership_tenure_years'] + 0.1)
    df['cost_per_year'] = df['total_claim_amount'] / (df['membership_tenure_years'] + 0.1)
    
    # Age-adjusted utilization
    age_avg_claims = df.groupby('age_group')['claims_per_year'].transform('mean')
    df['utilization_vs_age_cohort'] = df['claims_per_year'] / (age_avg_claims + 1)
    
    # === 2. CLAIM ACCELERATION SCORE ===
    # Are claims increasing over time?
    df['claim_acceleration_score'] = (
        (df['claims_count_6m'] / 6) / 
        ((df['claims_count_24m'] - df['claims_count_6m']) / 18 + 0.1)
    )
    df['claim_acceleration_score'] = df['claim_acceleration_score'].replace([np.inf, -np.inf], 0)
    
    # === 3. RECENT ACTIVITY SCORE ===
    df['recent_activity_score'] = (
        0.5 * (df['claims_count_3m'] / (df['total_claims_count'] + 1)) +
        0.3 * (df['total_amount_3m'] / (df['total_claim_amount'] + 1)) +
        0.2 * (1 / (df['days_since_last_claim'] + 30))
    )
    
    # === 4. SEVERITY RISK SCORE ===
    df['severity_risk_score'] = (
        0.4 * (df['high_cost_claims_count'] / (df['total_claims_count'] + 1)) +
        0.3 * (df['max_claim_amount'] / (df['avg_claim_amount'] + 1)) +
        0.3 * df['claim_amount_cv']
    )
    
    # === 5. CHRONICITY SCORE ===
    df['chronicity_score'] = (
        0.5 * df['has_high_risk_condition_history'] +
        0.3 * (df['chronic_conditions_count'] / (df['unique_condition_codes'] + 1)) +
        0.2 * (df['avg_days_between_claims'] < 60).astype(int)
    )
    
    # === 6. COMPOSITE AT-RISK SCORE (0-100) ===
    from sklearn.preprocessing import MinMaxScaler
    
    risk_components = [
        'utilization_vs_age_cohort',
        'claim_acceleration_score', 
        'recent_activity_score',
        'severity_risk_score',
        'chronicity_score'
    ]
    
    scaler = MinMaxScaler()
    df[risk_components] = scaler.fit_transform(df[risk_components])
    
    df['composite_risk_score'] = (
        0.25 * df['utilization_vs_age_cohort'] +
        0.20 * df['claim_acceleration_score'] +
        0.20 * df['recent_activity_score'] +
        0.20 * df['severity_risk_score']+
        0.15 * df['chronicity_score']
    ) * 100
    
    return df

risk_features = create_risk_score_features(claims_features, demographic_features)
risk_features[['Unique ID', 'composite_risk_score']].describe()

C:\Temp\ipykernel_15380\2188301879.py:23: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(dtype):
C:\Temp\ipykernel_15380\2188301879.py:42: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_avg_claims = df.groupby('age_group')['claims_per_year'].transform('mean')


,composite_risk_score
count,120000.000000
mean,4.377993
std,3.708574
min,0.327535
25%,3.200204
50%,3.200204
75%,3.200204
max,79.944555


In [57]:
demographic_features.sample(2)

,Unique ID,Year of Birth,age,age_group,gender_Female,gender_Male,gender_Other,Original Date of Joining,membership_tenure_days,membership_tenure_years,Registration Status,is_active_member,Scheme Category/ Section Name,contract_duration_days,Lapse Date,has_lapsed
26963,MEM00011877-01,1972,52,45-54,True,False,False,2018-08-18,2114,5.787817,Active,1,Platinum Plan,365,None,0
70960,MEM00031345-04,2005,19,<25,True,False,False,2021-09-13,992,2.715948,Active,1,Enhanced Plan,365,None,0


In [58]:
claims_features.sample(2)

,Unique ID,total_claims_count,total_claim_amount,avg_claim_amount,median_claim_amount,std_claim_amount,min_claim_amount,max_claim_amount,claims_count_3m,total_amount_3m,...,days_since_first_claim,days_since_last_claim,claim_duration_days,avg_days_between_claims,avg_monthly_claim_frequency,std_monthly_claim_frequency,max_monthly_claims,high_cost_claims_count,claim_amount_cv,claim_amount_trend
24916,MEM00040160-03,5,2522.59,504.518,239.52,652.050033,149.84,1668.34,0.0,0.00,...,1171,885,286,71.5,1.25,0.5,2,0,1.292422,0.053381
10956,MEM00017639-01,3,6821.43,2273.810,1809.18,2331.637829,209.47,4802.78,1.0,1809.18,...,436,73,363,181.5,1.00,0.0,1,1,1.025432,0.343044


In [59]:
treatment_condition_features.sample(2)

,Unique ID,unique_treatment_types,most_frequent_treatment,treatment_diversity_entropy,unique_condition_codes,unique_condition_categories,chronic_conditions_count,has_high_risk_condition_history,unique_treatment_locations,unique_providers
13257,MEM00021374-01,3,Specialist Consultation,1.039721,4,4,0.0,0,4,4
19516,MEM00031370-02,1,Specialist Consultation,0.000000,1,1,0.0,0,1,1


In [60]:
targets.sample(2)

,Unique ID,future_total_claims,will_make_high_claim,will_develop_high_risk_condition,is_at_risk,risk_severity_score
34310,MEM00015158-02,0.0,0,0,0,0.0
8225,MEM00003601-02,0.0,0,0,0,0.0


In [61]:
risk_features.sample(2)

,Unique ID,Year of Birth,age,age_group,gender_Female,gender_Male,gender_Other,Original Date of Joining,membership_tenure_days,membership_tenure_years,...,unique_treatment_locations,unique_providers,claims_per_year,cost_per_year,utilization_vs_age_cohort,claim_acceleration_score,recent_activity_score,severity_risk_score,chronicity_score,composite_risk_score
80590,MEM00035619-02,1980,44,35-44,False,True,False,2018-08-13,2119,5.801506,...,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.008259,0.000000,0.202335,3.200204
45052,MEM00019871-01,1982,42,35-44,False,True,False,2021-01-15,1233,3.375770,...,3.0,2.0,0.863118,11988.284539,0.000994,0.00072,0.449680,0.064358,0.505837,17.907542


In [64]:
risk_features.columns

Index(['Unique ID', 'Year of Birth', 'age', 'age_group', 'gender_Female',
       'gender_Male', 'gender_Other', 'Original Date of Joining',
       'membership_tenure_days', 'membership_tenure_years',
       'Registration Status', 'is_active_member',
       'Scheme Category/ Section Name', 'contract_duration_days', 'Lapse Date',
       'has_lapsed', 'total_claims_count', 'total_claim_amount',
       'avg_claim_amount', 'median_claim_amount', 'std_claim_amount',
       'min_claim_amount', 'max_claim_amount', 'claims_count_3m',
       'total_amount_3m', 'claims_count_6m', 'total_amount_6m',
       'claims_count_12m', 'total_amount_12m', 'claims_count_24m',
       'total_amount_24m', 'days_since_first_claim', 'days_since_last_claim',
       'claim_duration_days', 'avg_days_between_claims',
       'avg_monthly_claim_frequency', 'std_monthly_claim_frequency',
       'max_monthly_claims', 'high_cost_claims_count', 'claim_amount_cv',
       'claim_amount_trend', 'unique_treatment_types',
     

### 3.5 Merge All Features

In [68]:
def create_master_feature_set(risk_features, targets):
    """
    Combine all feature sets into final dataset
    """
    
    # Start with risk_features
    master_df = risk_features.copy()
    
    # Add targets
    master_df = master_df.merge(targets, on='Unique ID', how='left')

    
    
    print(f"Master dataset shape: {master_df.shape}")
    print(f"Features: {master_df.shape[1] - 5}")  # excluding ID and targets
    
    return master_df

master_df = create_master_feature_set(
    risk_features,
    targets
)

print("\nFeature Summary:")
print(master_df.dtypes.value_counts())
print(f"\nMissing values: {master_df.isnull().sum().sum()}")

Master dataset shape: (120000, 63)
Features: 58

Feature Summary:
float64           44
object             5
int32              5
int64              4
bool               3
category           1
datetime64[ns]     1
Name: count, dtype: int64

Missing values: 0


#### STEP 4: Data Preprocessing
##### 4.1 Handle Missing Values & Outliers

In [ ]:
 def preprocess_features(df):
    """
    Final preprocessing before modeling
    """
    
    # === 1. HANDLE MISSING VALUES ===
    print("Missing values before imputation:")
    print(df.isnull().sum()[df.isnull().sum() > 0])
    
    # Numerical: median imputation
    numerical_cols = df.select_dtypes(include=[np.number]).columns
    from sklearn.impute import SimpleImputer
    
    imputer = SimpleImputer(strategy='median')
    df[numerical_cols] = imputer.fit_transform(df[numerical_cols])
    
    # === 2. HANDLE OUTLIERS (CAP AT 99TH PERCENTILE) ===
    cost_columns = [col for col in df.columns if 'amount' in col.lower() or 'cost' in col.lower()]
    
    for col in cost_columns:
        if df[col].dtype in [np.float64, np.int64]:
            p99 = df[col].quantile(0.99)
            df[col] = df[col].clip(upper=p99)
    
    # === 3. HANDLE INFINITE VALUES ===
    df = df.replace([np.inf, -np.inf], 0)
    
    # === 4. ENCODE CATEGORICAL VARIABLES ===
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    
    # For high cardinality categoricals, use target encoding (do this during modeling)
    # For low cardinality, use one-hot encoding
    
    low_cardinality = []
    for col in categorical_cols:
        if df[col].nunique() <= 10:
            low_cardinality.append(col)
    
    df = pd.get_dummies(df, columns=low_cardinality, drop_first=True)
    
    print(f"\nAfter preprocessing: {df.shape}")
    
    return df

processed_df = preprocess_features(master_df)

Missing values before imputation:
Series([], dtype: int64)

After preprocessing: (120000, 68)


In [71]:
processed_df.columns

Index(['Unique ID', 'Year of Birth', 'age', 'gender_Female', 'gender_Male',
       'gender_Other', 'Original Date of Joining', 'membership_tenure_days',
       'membership_tenure_years', 'is_active_member',
       'Scheme Category/ Section Name', 'contract_duration_days', 'Lapse Date',
       'has_lapsed', 'total_claims_count', 'total_claim_amount',
       'avg_claim_amount', 'median_claim_amount', 'std_claim_amount',
       'min_claim_amount', 'max_claim_amount', 'claims_count_3m',
       'total_amount_3m', 'claims_count_6m', 'total_amount_6m',
       'claims_count_12m', 'total_amount_12m', 'claims_count_24m',
       'total_amount_24m', 'days_since_first_claim', 'days_since_last_claim',
       'claim_duration_days', 'avg_days_between_claims',
       'avg_monthly_claim_frequency', 'std_monthly_claim_frequency',
       'max_monthly_claims', 'high_cost_claims_count', 'claim_amount_cv',
       'claim_amount_trend', 'unique_treatment_types',
       'most_frequent_treatment', 'treatment_div

# 4.2 Feature Selection

In [75]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.ensemble import RandomForestClassifier

def select_important_features(df, target_col='is_at_risk', top_k=50):
    """
    Select most important features using variance filtering,
    univariate tests, and tree-based importance.
    Automatically handles datetime, bool, and categorical columns.
    """
    
    # --- 1. Separate features and target
    feature_cols = [col for col in df.columns if col not in [
        'Unique ID', 'is_at_risk', 'will_make_high_claim', 
        'will_develop_high_risk_condition', 'future_total_claims', 'risk_severity_score'
    ]]
    
    X = df[feature_cols].copy()
    y = df[target_col]

    # --- 2. Drop or encode unsupported dtypes
    # Convert bool → int, drop datetime/object/categorical
    drop_cols = []
    for col in X.columns:
        if np.issubdtype(X[col].dtype, np.datetime64):
            drop_cols.append(col)
        elif pd.api.types.is_bool_dtype(X[col]):
            X[col] = X[col].astype(int)
        elif pd.api.types.is_object_dtype(X[col]) or pd.api.types.is_categorical_dtype(X[col]):
            # Convert to category codes (or drop if too many unique)
            if X[col].nunique() < 50:
                X[col] = X[col].astype('category').cat.codes
            else:
                drop_cols.append(col)

    if drop_cols:
        print(f"Dropping non-numeric/unencoded columns: {drop_cols}")
        X = X.drop(columns=drop_cols)
    
    # Replace any remaining NaNs with 0 (safe for numeric)
    X = X.fillna(0)

    # --- 3. Remove low variance features
    selector = VarianceThreshold(threshold=0.01)
    X_high_var = selector.fit_transform(X)
    selected_features = X.columns[selector.get_support()].tolist()
    
    print(f"✅ Features after variance filter: {len(selected_features)}")

    # --- 4. Univariate feature selection (ANOVA F-test)
    X_selected = X[selected_features]
    selector = SelectKBest(f_classif, k=min(top_k, len(selected_features)))
    selector.fit(X_selected, y)
    
    feature_scores = pd.DataFrame({
        'feature': selected_features,
        'score': selector.scores_
    }).sort_values('score', ascending=False)
    
    print("\n📊 Top 20 features by F-score:")
    print(feature_scores.head(20))

    # --- 5. Random Forest feature importance
    rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
    rf.fit(X_selected, y)
    
    feature_importance = pd.DataFrame({
        'feature': selected_features,
        'importance': rf.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n🌳 Top 20 features by Random Forest importance:")
    print(feature_importance.head(20))

    # --- 6. Combine top features from both methods
    top_features_f = set(feature_scores.head(top_k)['feature'])
    top_features_rf = set(feature_importance.head(top_k)['feature'])
    final_features = list(top_features_f.union(top_features_rf))
    
    print(f"\n🏁 Final selected features: {len(final_features)}")
    
    return final_features, feature_importance



selected_features, feature_importance = select_important_features(processed_df)



C:\Temp\ipykernel_15380\691388711.py:30: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_object_dtype(X[col]) or pd.api.types.is_categorical_dtype(X[col]):
C:\Temp\ipykernel_15380\691388711.py:30: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_object_dtype(X[col]) or pd.api.types.is_categorical_dtype(X[col]):
C:\Temp\ipykernel_15380\691388711.py:30: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_object_dtype(X[col]) or pd.api.types.is_categorical_dtype(X[col]):
C:\Temp\ipykernel_15380\691388711.py:30: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.Cate

Dropping non-numeric/unencoded columns: ['Original Date of Joining', 'Lapse Date']
✅ Features after variance filter: 55

📊 Top 20 features by F-score:
                        feature        score
18              total_amount_3m  9040.486591
46        recent_activity_score  7524.030475
20              total_amount_6m  7227.962512
48         composite_risk_score  4844.308412
22             total_amount_12m  3605.454087
17              claims_count_3m  2160.144243
19              claims_count_6m  2136.104755
6       membership_tenure_years  2027.724421
5        membership_tenure_days  2027.724421
24             total_amount_24m  1479.041386
45                cost_per_year  1215.600697
21             claims_count_12m  1173.332588
42   unique_treatment_locations   916.852701
38       unique_condition_codes   894.300686
35       unique_treatment_types   873.148338
39  unique_condition_categories   816.538978
43             unique_providers   741.996197
37  treatment_diversity_entropy   728.6